In [6]:
import arkouda as ak
import arachne as ar
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import os
import time

ak.connect("n81", 5555)
p = 0.0005
num_nodes = 10_000

# Generate graph using NetworkX
graph = nx.gnp_random_graph(num_nodes, p, directed=True, seed=42)
src_list, dst_list = zip(*graph.edges())
src_ak = ak.array(src_list)
dst_ak = ak.array(dst_list)

start = time.time()
temp_prop_graph = ar.PropGraph()
end = time.time()
build_time = end - start
print("networkx graph generation....")
print(f"Building property graph with {num_nodes} vertices and "
      f"{len(src_list)} edges took {round(build_time,2)} seconds.")

### Generate node labels and edge relationships for the graph.
# 1. Extract node and edge information.
num_edges = len(src_list)
edges = (src_ak, dst_ak)
nodes = ak.arange(num_nodes)

##############################################
# NEW SECTION: Generate random node and edge attributes
##############################################

# For nodes:
# 2.1. Categorical from ["blue", "red", "yellow"]
# colors = ak.array(["blue", "red", "yellow"])
# node_colors = colors[ak.randint(0, len(colors), num_nodes)]

# 2.2. Integer from 10 to 12 (inclusive)
node_ints = ak.randint(10, 13, num_nodes)

# 2.3. Boolean from [False, True]
node_bools = ak.randint(0, 2, num_nodes, dtype=ak.bool)

# For edges:
# 2.4. Randomly choose from [5, 10]
edge_ints = ak.randint(5, 11, num_edges)

##############################################
# Create dataframes with the new attributes
##############################################

edge_df = ak.DataFrame({
    "src": edges[0],
    "dst": edges[1],
    "edge_attr": edge_ints
})

node_df = ak.DataFrame({
    "nodes": nodes,
    # "color": node_colors,
    "int_attr": node_ints,
    "bool_attr": node_bools
})

##############################################
# Create the new property graph with these attributes
##############################################

prop_graph = ar.PropGraph()
prop_graph.load_edge_attributes(edge_df, source_column="src", destination_column="dst")
prop_graph.load_node_attributes(node_df, node_column="nodes")
print("Property graph created with random node and edge attributes.")
print("num_edges = ", num_edges)
print("")



connected to arkouda server tcp://*:5555


networkx graph generation....
Building property graph with 10000 vertices and 49880 edges took 0.0 seconds.
Property graph created with random node and edge attributes.
num_edges =  49880



In [7]:
### Create the subgraph we are searching for.

src_list = [2, 3, 1, 3]
dst_list = [3, 1, 2, 0]

src_subgraph = ak.array(src_list)
dst_subgraph = ak.array(dst_list)

subgraph_nodes = list(set(src_list) | set(dst_list))
subgraph_nodes.sort()

# Generate random node and edge attributes for the subgraph
num_subgraph_nodes = len(subgraph_nodes)
num_subgraph_edges = len(src_list)

subgraph_node_ints = ak.array([11]*num_subgraph_nodes)
subgraph_node_bools = ak.array([False]*num_subgraph_nodes)
subgraph_edge_ints = ak.array([6]*num_subgraph_edges)
subgraph_edge_bools = ak.array([False]*num_subgraph_edges)

# Create dataframes for subgraph attributes
edge_df_h = ak.DataFrame({
    "src": src_subgraph,
    "dst": dst_subgraph,
    "rels1": subgraph_edge_ints,
    "rels2": subgraph_edge_bools
})

node_df_h = ak.DataFrame({
    "nodes": ak.array(subgraph_nodes),
    "lbls2": subgraph_node_ints,
    "lbls3": subgraph_node_bools
})

# Create the subgraph with these attributes
subgraph = ar.PropGraph()
subgraph.load_edge_attributes(edge_df_h, source_column="src", destination_column="dst")
subgraph.load_node_attributes(node_df_h, node_column="nodes")



print("Subgraph created with fixed node and edge attributes.")
print(" Run Arachne....")

Subgraph created with fixed node and edge attributes.
 Run Arachne....


In [8]:
"""VF2-SI """

isos_as_vertices = ar.subgraph_isomorphism(prop_graph, subgraph, 
                                           semantic_check = "and", algorithm_type = "si",
                                           reorder_type = "structural", return_isos_as = "vertices")

print(f"We found {len(isos_as_vertices[0])/len(subgraph)} monos inside of the graph")


# Print results
num_monos = len(isos_as_vertices[0]) / len(subgraph) if len(subgraph) > 0 else 0
print(f"We found {num_monos} monos inside of the graph.")

We found 0.0 monos inside of the graph
We found 0.0 monos inside of the graph.
